<a href="https://colab.research.google.com/github/srikrish2812/pytorch_practice/blob/main/torch_2_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PyTorch NN Classification